First find the embeddings of all words in the dataset and then pick a random word and find 10 words that are close to it.Your metric to find similarity should be **Euclidean Distance**.

#Load Pretrain Model

In [1]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tenso

In [2]:
import torch
import re
from transformers import BertModel, BertTokenizer

In [3]:
model_checkpoint = "bert-base-uncased"

In [4]:
model = BertModel.from_pretrained(model_checkpoint, output_hidden_states = True)
model.eval()
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def bert_text_preparation(text, tokenizer):
  """
  Preprocesses text input in a way that BERT can interpret.
  """
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1]*len(indexed_tokens)
  # convert inputs to tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensor = torch.tensor([segments_ids])
  return tokenized_text, tokens_tensor, segments_tensor

In [6]:
def get_bert_embeddings(tokens_tensor, segments_tensor, model):
  """
  Obtains BERT embeddings for tokens.
  """
  # gradient calculation id disabled
  with torch.no_grad():
    # obtain hidden states
    outputs = model(tokens_tensor, segments_tensor)
    hidden_states = outputs[2]
  # concatenate the tensors for all layers
  # use "stack" to create new dimension in tensor
  token_embeddings = torch.stack(hidden_states, dim=0)
  # remove dimension 1, the "batches"
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  # swap dimensions 0 and 1 so we can loop over tokens
  token_embeddings = token_embeddings.permute(1,0,2)
  # intialized list to store embeddings
  token_vecs = []
  # "token_embeddings" is a [Y x 12 x 768] tensor
  # where Y is the number of tokens in the sentence
  # loop over tokens in sentence
  for token in token_embeddings:
  # "token" is a [12 x 768] tensor
  # sum the vectors from the last four layers
      token_vec = token[-1]
      token_vecs.append(token_vec)
  return token_vecs

#Load Dataset and Preprocess

In [15]:
import pandas as pd
df = pd.read_csv("Q1_sent_train.csv")

In [16]:
df.head()

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0


In [17]:
def delete_hashtag_usernames(text):
  try:
    result = []
    for word in text.split():
      if word[0] not in ['@', '#']:
        result.append(word)
    return ' '.join(result)
  except:
    return ''

def delete_url(text):
  text = re.sub(r'http\S+', '', text)
  return text

In [18]:
import string 
special_tokens = ['[UNK]', '[CLS]', '[SEP]']
def remove_unuseful_tokens(tokens):
  tokens_without_stopwords = list()
  for token in context_sorted:
    if token in string.punctuation:
      continue
    elif '#' in token:
      continue
    elif token in special_tokens:
      continue
    elif token[0] == '[':
      continue
    else:
      tokens_without_stopwords.append(token)
  return tokens_without_stopwords

In [19]:
# 1. extract all tweets from files and save them in memory base on each year.

texts = df["text"]
normalized_list = []
for text in texts:
  new_text = delete_url(text)
  new_text = delete_hashtag_usernames(new_text)
  normalized_list.append(new_text)

normalized_texts = pd.DataFrame(normalized_list, columns=['text'])
tweets = normalized_texts["text"]

# 2. remove urls, hashtags and usernames.

In [20]:
sentences = tweets[:1000]
from collections import OrderedDict
context_dict = {} # key is token and values are (embeddings, count)
result_dict1 = dict()
context_embeddings = []
context_tokens = []
for sentence in sentences:
  tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(sentence, tokenizer)
  list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
  # make ordered dictionary to keep track of the position of each   word
  tokens = OrderedDict()
  # loop over tokens in sensitive sentence
  for token in tokenized_text[1:-1]:
    # keep track of position of word and whether it occurs multiple times
    if token in tokens:
      tokens[token] += 1
    else:
      tokens[token] = 1
    # compute the position of the current token
    token_indices = [i for i, t in enumerate(tokenized_text) if t == token]
    current_index = token_indices[tokens[token]-1]
    # get the corresponding embedding
    token_vec = list_token_embeddings[current_index]
    
    # save values
    if token in context_dict:
      context_dict[token]['embedding'] += token_vec
      context_dict[token]['count'] += 1
    else:
      context_dict[token] = {'embedding': token_vec, 'count': 1}

context_sorted = sorted(context_dict, key=lambda k: context_dict[k]['count'])
context_sorted.reverse()

tokens_without_stopwords = remove_unuseful_tokens(context_sorted)

for token in tokens_without_stopwords[:1000]:
  context_tokens.append(token)
  context_embeddings.append(context_dict[token]['embedding'] / context_dict[token]['count'])
  result_dict1[token] = context_dict[token]['embedding'] / context_dict[token]['count']

#Find 10 Nearest neighbor

In [26]:
def find_k_nearest_neighbors(word, embedding_dict, k):
  # Impelement Euclidean Distance and them find k nearest neighbors of word using this metric
  words_cosine_similarity = dict()
  for token in embedding_dict.keys():
    words_cosine_similarity[token] = - (embedding_dict[word] - embedding_dict[token]).pow(2).sum().sqrt().item()
  words_cosine_similarity = dict(sorted(words_cosine_similarity.items(), key=lambda item: item[1]))
  return list(words_cosine_similarity.keys())[-k:][::-1]


In [27]:
(result_dict1["assets"] - result_dict1["get"]).pow(2).sum().sqrt().item()

13.843334197998047

In [28]:
import random

In [24]:
len(list(result_dict1.keys()))

1000

In [ ]:
random_word = random.randint(0,1000)
tokens = list(result_dict1.keys())
word = tokens[random_word]
word

'divide'

In [29]:
random_word = random.randint(0,1000)
tokens = list(result_dict1.keys())
word = tokens[random_word] # Pick a random word from dataset
## alternative 
word = "divide"
# 2. find 10 nearest words
top_10_nearest_words = find_k_nearest_neighbors(word, result_dict1, 10)

# Print the top-10 words
for i, word in enumerate(top_10_nearest_words):
    print(f"{i}- {word}")

0- divide
1- share
2- buy
3- profit
4- investment
5- rate
6- dollar
7- cash
8- growth
9- gains
